In [117]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain.docstore.document import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableSequence,
    RunnableMap,
)
from langchain.memory import ConversationBufferMemory
from operator import itemgetter

# from langchain.retrievers import EnsembleRetriever


from kiwipiepy import Kiwi
from common import load_prompt
from dotenv import load_dotenv

load_dotenv()
kiwi = Kiwi()

In [118]:
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [119]:
DATA_DIR = "./data"

documents = []
for dir in os.listdir(DATA_DIR):
    file_dir = os.path.join(DATA_DIR, dir)
    if dir in [".DS_Store"]:
        continue
    for filename in os.listdir(file_dir):
        file_path = os.path.join(file_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            text = json.dumps(data, ensure_ascii=False)
            documents.append(
                {
                    "content": text,
                    "metadata": {
                        "file_path": file_path,
                        "info": {
                            "caseField": data.get("info", {}).get("caseFleid", ""),
                            "detailField": data.get("info", {}).get("detailField", ""),
                            "trailField": data.get("info", {}).get("trailField", ""),
                            "caseNm": data.get("info", {}).get("caseNm", ""),
                            "courtNm": data.get("info", {}).get("courtNm", ""),
                            "judmnAdjuDe": data.get("info", {}).get("judmnAdjuDe", ""),
                            "caseNo": data.get("info", {}).get("caseNo", ""),
                            "relateLaword": data.get("info", {}).get(
                                "relateLaword", ""
                            ),
                        },
                    },
                }
            )

In [120]:
# def kiwi_tokenize(text):
#     tokens = kiwi.tokenize(text)
#     return [token.form if hasattr(token, "form") else token for token in tokens]

In [121]:
docs = [
    Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in documents
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=10)
split_documents = text_splitter.split_documents(docs)


kiwi_bm25 = BM25Retriever.from_documents(
    split_documents,
    k=3,  # preprocess_func=kiwi_tokenize
)

# embeddings = OpenAIEmbeddings()
# faiss = FAISS.from_documents(
#     documents=split_documents, embedding=embeddings
# ).as_retriever(search_kwargs={"k": 3, "fetch_k": 10})

# kiwibm25_faiss_73 = EnsembleRetriever(
#     retrievers=[kiwi_bm25, faiss], weights=[0.7, 0.3], search_type="mmr"
# )

# retriever = kiwibm25_faiss_73

retriever = kiwi_bm25

In [122]:
def extract_context(inputs):
    results = retriever.invoke(inputs["question"])
    context = []
    for doc in results:
        context.append({"content": doc.page_content, "metadata": doc.metadata})
    return context

In [123]:
extract_chat_history = RunnableLambda(
    lambda inputs: memory.load_memory_variables(inputs)
) | itemgetter("chat_history")
extract_context = RunnableLambda(lambda inputs: retriever.invoke(inputs["question"]))
pass_question = RunnablePassthrough()
extract_metadata = RunnableLambda(
    lambda inputs: retriever.invoke(inputs["question"]).metadata
)

In [124]:
retriever.invoke(
    "피고인이 폭행으로 피해자에 대하여 성기에 손가락을 넣는 행위를 하였을 때 관련된 법에 대해 알려줘"
)[0]

Document(metadata={'file_path': './data\\2021\\2021고합169.json', 'info': {'caseField': '', 'detailField': '6', 'trailField': '1', 'caseNm': '유사강간', 'courtNm': '인천지방법원', 'judmnAdjuDe': '2021. 08. 20.', 'caseNo': '2021고합169', 'relateLaword': ['형법 제297조의2', '형법 제53조', '형법 제55조', '형법 제62조', '성폭력범죄의 처벌 등에 관한 특례법 제16조', '성폭력범죄의 처벌 등에 관한 특례법 제47조', '성폭력범죄의 처벌 등에 관한 특례법 제49조', '구 아동·청소년의 성보호에 관한 법률(2020. 5. 19. 법률 제17282호로 개정되기 전의 것)  제49조', '구 아동·청소년의 성보호에 관한 법률(2020. 5. 19. 법률 제17282호로 개정되기 전의 것) 제50조', '구 아동·청소년의 성보호에 관한 법률(2020. 5. 19. 법률 제17282호로 개정되기 전의 것) 제56조', '장애인복지법 제59조의3', '성폭력범죄의 처벌 등에 관한 특례법 제42조', '성폭력범죄의 처벌 등에 관한 특례법 제43조']}}, page_content='징역 1년에 처한다.", "이 판결 확정일부터 2년간 위 형의 집행을 유예한다.", "피고인에게 40시간의 성폭력 치료프로그램의 수강 및 80시간의 사회봉사를 명한다.", "피고인에게 아동,청소년 관련기관 등과 장애인복지시설에 각 3년간 취업제한을 명한다."]}, "mentionedItems": {"rqestObjet": ["피고인은 폭행으로 피해자에 대하여 성기에 손가락을 넣는 행위를 하였다."]}, "assrs": {"acusrAssrs": [], "dedatAssrs": []}, "facts": {"bsisFacts": ["피고인은 피해자 B(여, 19세)와 지인 사이이다.", "피고인은 2020.11

In [125]:
prompt = load_prompt("./prompts/prompt.yaml", encoding="utf-8")

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

chain = prompt | llm | StrOutputParser()
runnable_map = RunnableMap(
    {
        "chat_history": extract_chat_history,
        "context": extract_context,
        "question": pass_question,
    }
)
runnable_chain = RunnableSequence(runnable_map, chain)

In [130]:
response = runnable_chain.invoke(
    {
        "question": "피고인이 폭행으로 피해자에 대하여 성기에 손가락을 넣는 행위를 하였을 때 관련된 법에 대해 알려줘"
    }
)
print(response)

형법 제297조의2에 따르면, 피고인이 폭행으로 피해자에 대하여 성기에 손가락을 넣는 행위는 유사강간에 해당합니다.


In [131]:
runnable_chain.invoke({"question": "방금 물어본 내용과 관련된 사건 번호를 모두 알려줘"})

'2012고단3562, 2013고단2813, 2011고합12'

In [132]:
runnable_chain.invoke(
    {"question": "방금 물어본 내용에서 사건을 담당한 법원을 모두 알려줘"}
)

'대전지방법원, 서울동부지방법원'

In [133]:
runnable_chain.invoke(
    {"question": "방금 물어본 내용과 관련된 판결 선고일을 모두 알려줘"}
)

'2013. 05. 24., 2014. 06. 18., 2014. 03. 13.'

In [141]:
json_string = json.dumps(documents[0].get("content"), ensure_ascii=False)
data = json.loads(json.loads(json_string)).get("facts").get("bsisFacts")
print(data)

['피고인은 공소외 00, 00, 00, 00, 00과 공모하여 피고인은 형광칩을 삽입하여 패의 투시가 가능하도록 한 특수화투를 제작하고, 위 00은 카메라, 노트북컴퓨터, 화면카트기 등 화투투시용 기기 20여 종을 준비하고, 위 00, 00는 몰래 카메라를 조작하면서 화투의 패를 읽고, 위 00는 직접 이른바 선수로 참여하여 화투를 하였다.', '위 00, 00은 이른바 선수를 모집하기로 역할을 분담하여, 1. 2000. 3. 10. 15:00경부터 같은 날 21:00경까지 사이에 부산 00구 00동에 있는 00모텔 502호실에서, 위 00, 00, 00는 위 모텔 502호실 및 403호실을 빌려 502호실 에어컨 속에 소형 카메라를 설치하고, 403호실에 이 카메라를 통하여 도박현장을 볼 수 있도록 모니터, 노트북컴퓨터, 화면카트기 등을 설치하여 위 카메라와 연결하여 놓았다.', '위 00, 00은 사기도박을 할 상대로 피해자 00, 00, 00를 모집하여 온 다음, 위 00은 사기도박을 할 상대로 피해자 00, 00, 00를 모집하여 온 다음, 위 00가 사실은 위와 같이 파개 투시되는 특수화투를 이용하여 승패를 조작할 것임에도 불구하고 마치 우연하게 승부가 결정되는 것인 양 행세하였다.', '위 피해자들과 속칭 도리짓고땡이라는 도박을 하면서, 위 00, 00이 위 카메라로 바닥에 깔린 화투를 투시하여 패를 읽은 뒤 00가 소지하고 잇는 신호기로 신호를 보내고, 00는 그 신호에 따라 패를 확인하는 수법으로 사기도박을 하여 피해자 00로부터 500만 원을, 같은 00으로부터 500만 원을, 같은 00로부터 1,000만 원을, 각 따서 이를 편취하였다.', '2000. 3. 14. 15:00경부터 같은 날 22:00경까지 사이에 같은 장소에서, 피해자 00, 00, 00를 상대로 제1항 기재와 같은 방법으로 사기도박을 하여 피해자 00로부터 600만 원을, 같은 00로부터 300만 원을, 각 따서 이를 편취하였다.', '2000. 3. 15. 14:00경부

In [142]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


def make_question_data(documents):
    dataset_prompt = PromptTemplate.from_template(
        """
        사건 내용을 요약하여 사건 내용과 관련된 법이 어떤게 있는지 질의해주세요.
        한글로 문장을 요약해주어야 하며, 요약할 때 "질문:"을 붙이지 않고 요약해주세요.

        Question: {question}
        """
    )

    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

    dataset_chain = dataset_prompt | llm | StrOutputParser()

    json_string = json.dumps(documents.get("content"), ensure_ascii=False)
    data = json.loads(json.loads(json_string)).get("facts").get("bsisFacts")
    question_data = dataset_chain.invoke({"question": str(data)})

    return question_data

In [143]:
def ask_question(inputs):
    query_text = inputs.get("question", "")  # 'question' 키에 해당하는 값을 사용
    return {"answer": chain.invoke(query_text)}

In [144]:
questions = []
answer = []
for document in documents[:10]:
    temp_data = make_question_data(document)
    questions.append(temp_data)

print(questions)

['피고인은 공범들과 함께 특수한 화투와 투시 장비를 이용하여 사기도박을 계획하고 실행하였다. 이들은 모텔에 카메라와 모니터 등을 설치하여 도박 현장을 감시하고, 피해자들을 속여 도박을 진행하며 총 4,400만 원을 편취하였다. 마지막 시도는 경찰 단속으로 미수에 그쳤다. 관련 법으로는 사기죄와 도박죄가 있다.', '피고인들은 광산노조의 회계 및 자금 관리 업무를 담당하면서, 노조 자금을 개인 용도로 횡령하였습니다. 피고인 00은 노조 자금을 자신의 계좌로 이체하여 생활비, 아들 유학비, 부동산 구입 자금 등으로 사용하였고, 피고인 00과 공모하여 노조 자금을 변호사 선임 비용 등으로 사용하였습니다. 또한, 부가가치세 환급금을 포함한 노조 자금을 개인적으로 소비하였으며, 노조 명의 계좌에 입금된 자금을 빼돌려 아파트 분양대금 및 주식 구입 자금으로 사용하였습니다. 관련 법으로는 횡령죄가 적용될 수 있습니다.', '피고인 00는 수입산 쇠고기, 돼지고기, 닭고기를 국내산 또는 호주산으로 허위 표기하여 납품하고, 부정한 청탁을 받고 납품 경쟁 업체들의 견적서를 미리 알려주는 등의 방법으로 총 1,189,957,809원의 매출을 올렸으며, 여러 명의 피고인들이 부정한 청탁을 받고 금품을 수수하였다. 관련 법으로는 원산지 표시 위반, 부정 청탁 및 금품 등 수수의 금지에 관한 법률 등이 있을 수 있다.', '피고인은 피해 회사의 상무이사로 근무하며 횡령된 금원을 회수하는 업무를 담당하던 중, 회사 대표로부터 명의신탁을 받아 보관 중이던 아파트에 대해 임의로 근저당권을 설정하여 횡령하였다. 관련 법으로는 형법상 횡령죄가 적용될 수 있다.', '피고인들은 폭력조직의 일원으로, 경쟁업체 직원들과의 주차 문제로 다툼이 발생하자 폭력을 행사하여 상해를 입혔으며, 구치소 내에서도 피해자들에게 협박과 강요를 통해 의무 없는 일을 시켰다. 관련 법률로는 폭력행위등처벌에관한법률, 형법상 상해죄 및 협박죄 등이 있다.', '피고인은 아파트 관리사무소에서 경리 및 관리 업무를 담당하던 

In [145]:
answer = []
for document in documents[:10]:
    answer.append(document.get("metadata").get("info").get("relateLaword"))

print(answer)

[['형법 제347조', '형법 제352조', '형법 제30조 ', '형법 제37조', '형법 제38조', '형법 제50조', '형법 제62조', '형법 제48조'], ['형법 제356조', '형법 제355조', '형법 제30조', '형법 제37조', '형법 제38조', '형법 제50조', '형법 제70조', '형법 제69조', '형법 제62조', '형사소송법 제325조'], ['농수산물의 원산지 표시에 관한 법률 제14조', '농수산물의 원산지 표시에 관한 법률 제6조', '형법 제357조', '농수산물의 원산지 표시에 관한 법률 제17조', '형법 제37조', '형법 제38조', '형법 제50조', '형법 제62조', '형사소송법 제334조'], ['형법 제355조'], ['폭력행위 등 처벌에 관한 법률 제2조', '형법제 257조', '형법제324조', '형법 제30조', '형법 제37조', '형법 제39조', '형법 제38조', '형법 제50조'], ['형법 제356조', '형법 제355조', '형법 제231조', '형법 제234조', '형법 제37조', '형법 제38조', '형법 제50조', '형사소송법 제325조'], ['형법 제185조', '형법 제30조', '형법 제70조', '형법 제69조', '형사소송법 제334조', '집회 및 시위에 관한 법률 제22조', '집회 및 시위에 관한 법률 제5조', '형사소송법 제325조'], ['형법 제19조', '형법 제37조', '형법 제39조', '형사소송법 제325조'], ['형법 제9조', '소년법 제50조', '소년법 제58조', '소년법 제2조'], ['특정범죄 가중처벌 등에 관한 법률 제5조', '형법 제329조', '구 형법(2010. 4. 15. 법률 제10259호로 개정되기 전의 것) 제42조', '형법 제35조', '형법 제53조', '형법 제55조']]


In [146]:
import pandas as pd

df = pd.DataFrame({"question": questions, "answer": answer})
df["answer"] = df["answer"].apply(
    lambda x: str(x).replace("[", "").replace("]", "").replace("'", "")
)
df = df.iloc[:10, :]

In [147]:
from langsmith import Client

client = Client()
dataset_name = "LEGAL_CASE_EVAL_DATASET"


# 데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset

In [148]:
# 데이터셋 생성
dataset = create_dataset(client, dataset_name)

# 생성된 데이터셋에 예제 추가
client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)

In [149]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

In [152]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# 최종 판결을 결정하기 전에 LLM의 추론을 사용하도록 지시
qa_evalulator = LangChainStringEvaluator("cot_qa")

# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)

You are a teacher grading a quiz.
You are given a question, the context the question is about, and the student's answer. You are asked to score the student's answer as either CORRECT or INCORRECT, based on the context.
Write out in a step by step manner your reasoning to be sure that your conclusion is correct. Avoid simply stating the correct answer at the outset.

Example Format:
QUESTION: question here
CONTEXT: context the question is about here
STUDENT ANSWER: student's answer here
EXPLANATION: step by step reasoning here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
CONTEXT: {context}
STUDENT ANSWER: {result}
EXPLANATION:


In [153]:
dataset_name = "LEGAL_CASE_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="LEGAL_CASE_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "QA Evaluator 를 활용한 평가",
    },
)

View the evaluation results for experiment: 'LEGAL_CASE_EVAL-257d01e8' at:
https://smith.langchain.com/o/afbb9fbc-1061-4288-872f-46ef447d21c3/datasets/91b28711-c66b-4cec-b512-702f9e93e073/compare?selectedSessions=96298e84-04dc-40a1-8c77-bfebdcfe5824




10it [01:06,  6.68s/it]
